In [1]:
# libraries
import pandas as pd

In [2]:
df_clean = pd.read_csv("clean_march_randomness.csv")
df_clean = df_clean.loc[:,"rank":]

In [3]:
# create a scenario outcome vector

true_outcome = ["No", "Yes", "No", "Yes",
                "No", "No", "No", "Yes",
                "Yes", "No", "Yes", "No",
                "Yes", "Yes", "No", "Yes"]

In [4]:
# create and insert columns for StatCoin totals over time
df = df_clean

for num, boolean in enumerate(true_outcome):
    if num == 0:
        today_prefix = "day_" + str(num+1) + "_"
        correct_series = df[df[today_prefix + "guess"] == boolean][today_prefix + "bet"] + 4000
        
        # https://chartio.com/resources/tutorials/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe/
        no_answer_df = df[df[today_prefix + "guess"].isnull()]
        no_answer_df = no_answer_df.copy()
        no_answer_df.loc[:, today_prefix + "bet"] = 4000
        no_answer_series = no_answer_df[today_prefix + "bet"]
        
        not_wrong_index = list(no_answer_series.index) + list(correct_series.index)
        wrong_df = df.drop(not_wrong_index)
        wrong_normal_series = -1 * wrong_df[wrong_df[today_prefix + "bet"].isin([100,300,800])][today_prefix + "bet"] + 4000
        wrong_free_bet_series = -1 * wrong_df[wrong_df[today_prefix + "bet"] == 1500][today_prefix + "bet"] + 5500
        
        
        day_total = pd.concat([correct_series, no_answer_series, wrong_normal_series, wrong_free_bet_series]).sort_index()
        df.insert(loc=8+num*2, column = today_prefix + "total", value=day_total)
        
    elif num > 0:
        today_prefix = "day_" + str(num+1) + "_"
        yesterday_prefix = "day_" + str(num) + "_"
        correct_series = df[df[today_prefix + "guess"] == boolean][today_prefix + "bet"]
        
        # I learned about the `isnull()` function from the following link:
        # https://chartio.com/resources/tutorials/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe/
        no_answer_df = df[df[today_prefix + "guess"].isnull()]
        no_answer_df = no_answer_df.copy()
        no_answer_df.loc[:, today_prefix + "bet"] = 0
        no_answer_series = no_answer_df[today_prefix + "bet"]
        
        not_wrong_index = list(no_answer_series.index) + list(correct_series.index)
        wrong_df = df.drop(not_wrong_index)
        wrong_normal_series = -1 * wrong_df[wrong_df[today_prefix + "bet"].isin([100,300,800])][today_prefix + "bet"]
        wrong_free_bet_series = -1 * wrong_df[wrong_df[today_prefix + "bet"] == 1500][today_prefix + "bet"] + 1500
        
        
        day_changes = pd.concat([correct_series,
                                 no_answer_series,
                                 wrong_normal_series,
                                 wrong_free_bet_series]).sort_index()
        day_total = day_changes + df[yesterday_prefix + "total"]
        df.insert(loc=8+num*3, column = today_prefix + "total", value=day_total)

df

,rank,team_name,final_StatCoin_balance,bet_each_round,edu_level,competitor_type,day_1_guess,day_1_bet,day_1_total,day_2_guess,...,day_13_total,day_14_guess,day_14_bet,day_14_total,day_15_guess,day_15_bet,day_15_total,day_16_guess,day_16_bet,day_16_total
0,1,A Fraction Ahead,9000,yes,Pre-College,Team,Yes,300.0,3700.0,No,...,7300.0,Yes,1500.0,8800.0,No,100.0,8900.0,Yes,100.0,9000.0
1,2,Elidia R,8900,yes,Pre-College,Individual,Yes,1500.0,4000.0,No,...,6500.0,Yes,800.0,7300.0,No,800.0,8100.0,Yes,800.0,8900.0
2,3,Mathster's Stupefying 1 Stars,8400,yes,Pre-College,Team,Yes,800.0,3200.0,No,...,7100.0,No,300.0,6800.0,No,800.0,7600.0,Yes,800.0,8400.0
3,4,Dylan M,8000,yes,Pre-College,Individual,Yes,800.0,3200.0,No,...,5600.0,Yes,800.0,6400.0,No,800.0,7200.0,Yes,800.0,8000.0
4,4,J Fly Money,8000,yes,Pre-College,Individual,Yes,800.0,3200.0,No,...,7200.0,No,800.0,6400.0,No,800.0,7200.0,Yes,800.0,8000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,120,JCP,1600,no,Pre-College,Team,Yes,800.0,3200.0,No,...,3200.0,NaN,NaN,3200.0,Yes,800.0,2400.0,No,800.0,1600.0
210,120,Reject the Null,1600,no,Pre-College,Team,Yes,800.0,3200.0,No,...,600.0,Yes,300.0,900.0,No,1500.0,2400.0,No,800.0,1600.0
211,120,Reuben,1600,no,Pre-College,Individual,Yes,800.0,3200.0,No,...,1600.0,NaN,NaN,1600.0,NaN,NaN,1600.0,NaN,NaN,1600.0
212,123,Fisher 302,1400,no,College,Team,Yes,300.0,3700.0,NaN,...,1400.0,NaN,NaN,1400.0,NaN,NaN,1400.0,NaN,NaN,1400.0


In [5]:
df.to_csv("transformed_march_randomness.csv")

In [6]:
df_bet_each_round = df_bet_each_round = df[df["bet_each_round"] == "yes"]
df_bet_each_round

,rank,team_name,final_StatCoin_balance,bet_each_round,edu_level,competitor_type,day_1_guess,day_1_bet,day_1_total,day_2_guess,...,day_13_total,day_14_guess,day_14_bet,day_14_total,day_15_guess,day_15_bet,day_15_total,day_16_guess,day_16_bet,day_16_total
0,1,A Fraction Ahead,9000,yes,Pre-College,Team,Yes,300.0,3700.0,No,...,7300.0,Yes,1500.0,8800.0,No,100.0,8900.0,Yes,100.0,9000.0
1,2,Elidia R,8900,yes,Pre-College,Individual,Yes,1500.0,4000.0,No,...,6500.0,Yes,800.0,7300.0,No,800.0,8100.0,Yes,800.0,8900.0
2,3,Mathster's Stupefying 1 Stars,8400,yes,Pre-College,Team,Yes,800.0,3200.0,No,...,7100.0,No,300.0,6800.0,No,800.0,7600.0,Yes,800.0,8400.0
3,4,Dylan M,8000,yes,Pre-College,Individual,Yes,800.0,3200.0,No,...,5600.0,Yes,800.0,6400.0,No,800.0,7200.0,Yes,800.0,8000.0
4,4,J Fly Money,8000,yes,Pre-College,Individual,Yes,800.0,3200.0,No,...,7200.0,No,800.0,6400.0,No,800.0,7200.0,Yes,800.0,8000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,86,3 StDev Above Mu,2000,yes,Pre-College,Team,Yes,300.0,3700.0,No,...,1200.0,No,800.0,400.0,No,800.0,1200.0,Yes,800.0,2000.0
86,87,The Fluffy-Floofy Bears,1800,yes,College,Individual,Yes,1500.0,4000.0,No,...,2600.0,No,800.0,1800.0,No,800.0,2600.0,No,800.0,1800.0
87,88,The Gram,1600,yes,Pre-College,Team,Yes,800.0,3200.0,No,...,2400.0,Yes,800.0,3200.0,Yes,800.0,2400.0,No,800.0,1600.0
88,89,silly snakes,1400,yes,Pre-College,Team,Yes,800.0,3200.0,No,...,1700.0,No,300.0,1400.0,No,800.0,2200.0,No,800.0,1400.0


In [7]:
df_bet_each_round.to_csv("bet_each_round_march_randomness.csv")